In [1]:
from modules.dataloader import XView3DDN
import numpy as np
from torch.utils.data import DataLoader
import torch
import torchvision
from torch.utils.tensorboard import SummaryWriter

In [2]:
for i in range(torch.cuda.device_count()):
    print(i, torch.cuda.get_device_name(i))

0 TITAN Xp
1 TITAN X (Pascal)
2 Quadro K420


/home/xview3/anaconda3/envs/xview3/lib/python3.9/site-packages/torch/cuda/__init__.py:81: UserWarning: 
    Found GPU2 Quadro K420 which is of cuda capability 3.0.
    PyTorch no longer supports this GPU because it is too old.
    The minimum cuda capability that we support is 3.5.
    
  warnings.warn(old_gpu_warn % (d, name, major, capability[1]))


In [3]:
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
device0 = torch.device("cuda:1")
# device = torch.device("cpu")

## Model Preparation

In [4]:
from torchvision.models import resnet50
from torchvision import io, transforms as T

transform = T.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

# img = io.read_image("./lion.jpg")/255
# img = transform(img).reshape(1,3,224,224)
# img = img.to(device)

In [5]:
from modules.featextract import FeatureExtractor
from modules.model import RPN, DN
from modules.config import load_model_config
from modules.utils import evaluate, save_fig, convert_prob_to_image
from modules.losses import loss_fn_dn_2

In [6]:
def prepare_backbone():
    backbone = resnet50(pretrained=True)
    for param in backbone.parameters():
        param.requires_grad = False
    backbone = backbone.eval()
    
    return backbone

## Load RPN

In [7]:
rpn = torch.nn.Sequential(*[FeatureExtractor(load_model_config(prepare_backbone())), RPN(128)])
rpn.load_state_dict(torch.load("/media/xview/xview3_challenge/xView3-Challenge/exps/ckpts/RPN_2_trained_model_3_epochs.pth"))

for param in rpn.parameters():
    param.requires_grad = False

rpn.eval()
rpn.to(device0)
print()

In [8]:
def rpn_callable(x):
#     x = transform(x)
#     return rpn(x.to(device0)).to(torch.device("cpu"))
    return torch.ones_like(x)

## Load DN

In [9]:
model = torch.nn.Sequential(*[FeatureExtractor(load_model_config(prepare_backbone())), DN(128, 4)])
model.load_state_dict(torch.load("/media/xview/xview3_challenge/xView3-Challenge/exps/ckpts/DN_3_trained_model_3_epochs.pth"))

<All keys matched successfully>

## Dataloading

In [10]:
exp_no = 4

In [11]:
def preprocess_label(df):
    df = df.dropna(subset=["is_vessel"])
#     df = df[df["distance_from_shore_km"] <= 2]
    return df

train_data_path = "/media/xview/xview3_challenge/dataset/data/train"
train_label_path = "/media/xview/xview3_challenge/dataset/labels/train.csv"

val_data_path = "/media/xview/xview3_challenge/dataset/data/validation"
val_label_path = "/media/xview/xview3_challenge/dataset/labels/validation.csv"

# data_path = [train_data_path, val_data_path]
# label_path = [train_label_path, val_label_path]

train_data = XView3DDN(model_callable=rpn_callable, background_chip_ratio=0.1, obj_size=5, threshold=0.25, overwrite=True,
                        labels_path=train_label_path, data_path=train_data_path, preprocess_label=preprocess_label)


val_data = XView3DDN(model_callable=rpn_callable, background_chip_ratio=1.1, obj_size=5, threshold=0.25, overwrite=True,
                        labels_path=val_label_path, data_path=val_data_path, preprocess_label=preprocess_label)

# data = XView3Data(background_chip_ratio=0.1, obj_size=5, threshold=0.25, overwrite=False,
#                         labels_path=label_path, data_path=data_path, preprocess_label=preprocess_label)


Total scenes detected: 553
	Processing scene: 8325eea6c95aa16at
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: ced739d78afc3286t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: b95ab45640844f93t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: d3cf076c5f02d411t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: e9b02e333e21456ft
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 3225958292ca7026t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: d1e05813c04918a7t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		

	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 28faafcae2ac2ea9t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: c0831dbd6d7f3c56t
	Saving in labels_close_to_shore.json
	Total chips extracted: 3
		Background chips: 1
		Foreground chips: 2
	Processing scene: a4e67518374415e0t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 34f6ce4f7673ea6at
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: ba737e36cc3bbbd6t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 1185066a6428bc12t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: d89

	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 0229f98a344d7a5ct
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 90c677f83c4e18a9t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 1b576e404a94347ft
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 060add30418568b9t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 2ae2b77095e9e6eat
	Saving in labels_close_to_shore.json
	Total chips extracted: 3
		Background chips: 1
		Foreground chips: 2
	Processing scene: 0791adb6b1cef4f1t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 981

	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 38cfc7bccd4a425ft
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 2ef89b1ed1b11bc6t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 1a28df2c4f294a32t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 0e89decdecef87f6t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: eb7b92263f79f93et
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: db68648691defa60t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: bc3

	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 6f3e6b05cf9d127at
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 560196fa56af9f03t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: e98f90071c73034dt
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 70f29a17a192ae59t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: eacb90c9b9b80205t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 7a536b98f97647fft
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 14b

...

	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 4c4af230613862fat
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: a1dfd3c832f17b5et
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 5f2c3095128d4642t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 42b157923b7a27e5t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 896ce26722571c56t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 0edb4a514cda2059t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 9f4

	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 50f39eee83812d56t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: aa10fadb6bc7eb21t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 0aba50e606eddffet
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 91b3ca2616525cd3t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 6ffc913c4655b43ft
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 10615061cd28e01bt
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: eba

	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: ced7ae4f740a9eb7t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 741e6a842c23cdadt
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: bd24c2e8d539f972t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 7990e9589b9ff3bft
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: bd107d8b11227e84t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 7ef67f5bd2f89841t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: e20

	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 893b12c617265261t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 0d7a0261c60d39det
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: d546b16a2e492dd9t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 874dd7fe8f2c9d51t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 8ea7a53ff471dcc0t
	Saving in labels_close_to_shore.json
	Total chips extracted: 0
		Background chips: 0
		Foreground chips: 0
	Processing scene: 923ff111568e0d18t
	Saving in labels_close_to_shore.json
	Total chips extracted: 2
		Background chips: 1
		Foreground chips: 1
	Processing scene: 5b0

	Saving in labels_close_to_shore.json
	Total chips extracted: 19
		Background chips: 10
		Foreground chips: 9
	Processing scene: acf76647ba3f524bv
	Saving in labels_close_to_shore.json
	Total chips extracted: 126
		Background chips: 66
		Foreground chips: 60
	Processing scene: 65d6f6baee882077v
	Saving in labels_close_to_shore.json
	Total chips extracted: 32
		Background chips: 17
		Foreground chips: 15
	Processing scene: fc2f57371370c521v
	Saving in labels_close_to_shore.json
	Total chips extracted: 40
		Background chips: 21
		Foreground chips: 19
	Processing scene: 0d8ed29b0760dc59v
	Saving in labels_close_to_shore.json
	Total chips extracted: 78
		Background chips: 41
		Foreground chips: 37
	Processing scene: f9eb760aaf6e798dv
	Saving in labels_close_to_shore.json
	Total chips extracted: 171
		Background chips: 90
		Foreground chips: 81
	Processing scene: fe6a8d80fb5ebb8ev
	Saving in labels_close_to_shore.json
	Total chips extracted: 116
		Background chips: 61
		Foreground chips: 55

In [12]:
batch_size = 8
n_epochs = 50

In [13]:
train_sampler = torch.utils.data.RandomSampler(train_data)
val_sampler = torch.utils.data.RandomSampler(val_data)
# data_sampler = torch.utils.data.RandomSampler(data)


data_loader_train = torch.utils.data.DataLoader(train_data, batch_size=batch_size, sampler=train_sampler, num_workers=0, pin_memory=True)
data_loader_val = torch.utils.data.DataLoader(val_data, batch_size=batch_size, sampler=val_sampler, num_workers=0, pin_memory=True)

In [14]:
writer = SummaryWriter("/media/xview/xview3_challenge/xView3-Challenge/exps/runs/factseg_experiment_DN"+str(exp_no))

In [15]:
# model.train()
model.to(device)
optimizer = torch.optim.SGD(model.parameters(), lr=0.0001, weight_decay=0.0001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.9)

class_weight = torch.Tensor([0.1, 1, 1, 1]).to(device)

In [16]:
def loss_fn_cal(*args, **kwargs):
#     kwargs["class_weight"] = class_weight
    return loss_fn_dn_2(args[1], args[2])

In [17]:
running_loss = 0

for epoch in range(n_epochs):
    num_nans = 0
    print(f"\nEpoch: {epoch+1}")
    for i, data in enumerate(data_loader_train, 1):
        
        proposal_t, un_img, class_labels, inst_weight = data
        img = transform(un_img)
        img = img.to(device)
        
        class_labels = class_labels.to(device)
        
        
        optimizer.zero_grad()
        
        pred = model(img)

        loss = loss_fn_dn_2(pred, class_labels)
        
        print(f"\rStep: {i} Training Loss: {loss.item()} Validation Loss: {None} Nans: {num_nans}", end="")
        
        if torch.isnan(loss):
            if num_nans > 10:
                raise RuntimeError(f"Model Error: Encountered {num_nans} nan loss")
            num_nans += 1
            continue
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.detach().item()
        
        num_nans = 0
        if i%10 == 0:
            
            writer.add_scalar('training_loss', running_loss/15, epoch*len(data_loader_train)+i)
            
            step = None
            
            if i%10 == 0:
                step = epoch*len(data_loader_train)+i
                save_fig(writer, "train", step, un_img[0], pred[0], class_labels[0], multiclass=True)
                
            val_loss = evaluate(lambda x: model(x), data_loader_val, loss_fn_cal, step, 10, device, writer, transform=transform, multiclass=True)
            
            writer.add_scalar('validation_loss', val_loss, epoch*len(data_loader_train)+i)
            
            print(f"\rTraining Loss: {loss.item()} Validation Loss: {val_loss}")
            
            running_loss = 0.0
    
    if (epoch+1)%5 == 0:
        scheduler.step()
    
    checkpoint_pth = f'/media/xview/xview3_challenge/xView3-Challenge/exps/ckpts/DN_{exp_no}_trained_model_{epoch+1}_epochs.pth'
    torch.save(model.state_dict(), checkpoint_pth)

writer.close()


Epoch: 1
Training Loss: 1.8396382331848145 Validation Loss: 5.303295278549195s: 0
Training Loss: 2.261882781982422 Validation Loss: 5.091369652748108s: 00
Training Loss: 2.2792885303497314 Validation Loss: 4.681292915344239s: 0
Training Loss: 1.906240701675415 Validation Loss: 4.828519010543824s: 00

Epoch: 2
Training Loss: 1.723419427871704 Validation Loss: 4.909809207916259s: 0
Training Loss: 2.130708694458008 Validation Loss: 5.4511054992675785: 00
Training Loss: 1.5353500843048096 Validation Loss: 5.267223834991455s: 0
Training Loss: 1.8012685775756836 Validation Loss: 5.472728204727173s: 0

Epoch: 3
Training Loss: 1.6041845083236694 Validation Loss: 5.142165422439575s: 0
Training Loss: 1.7353699207305908 Validation Loss: 4.854635429382324s: 0
Training Loss: 1.5399106740951538 Validation Loss: 4.667108058929443s: 0
Training Loss: 1.968856692314148 Validation Loss: 5.303648614883423s: 00

Epoch: 4
Training Loss: 1.4960434436798096 Validation Loss: 5.622308158874512s: 0
Training Los